In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import jax
jax.config.update("jax_enable_x64", True)

In [2]:
import jax.numpy as jnp
from jax import jit, grad, jacobian, lax

import numpy as np
import matplotlib.pyplot as plt

In [60]:
n = 10

h0 = jnp.array(np.diag(np.sort(np.pi*np.random.uniform(size=n))))
mraw = np.random.normal(size=n**2).reshape((n,n)) + 1j*np.random.normal(size=n**2).reshape((n,n))
m = jnp.array(0.5*(mraw + mraw.conj().T))

print(jnp.mean(jnp.abs(h0 - h0.T)))
print(jnp.mean(jnp.abs(m - m.conj().T)))

0.0
0.0


In [61]:
dt = 1e-1

In [62]:
numsteps = 100

In [63]:
# given initial condition and forcing f, return trajectory a
def propSchro(f, a0):
    a = [a0]
    for k in range(numsteps):
        thisham = h0 + f[k]*m
        evals, evecs = jnp.linalg.eigh(thisham)
        a.append( evecs @ jnp.diag(jnp.exp(-1j*dt*evals)) @ evecs.conj().T @ a[k] )
    
    a = jnp.stack(a, axis=0)
    return a

In [64]:
# given forcing f, IC a0, FC alpha, return cost
rho = 1
def cost(f, a0, alpha):
    a = propSchro(f, a0)
    resid = a[-1] - alpha
    pen = jnp.real(jnp.sum(resid * resid.conj()))
    return 0.5*jnp.sum(f**2) + 0.5*rho*pen

In [65]:
mya0 = jnp.eye(n)[0]

In [66]:
a = propSchro( jnp.array(np.random.normal(size=numsteps)), mya0 )

In [67]:
myalpha = jnp.array(np.random.normal(size=n))

In [68]:
cost( jnp.array(np.random.normal(size=numsteps)), mya0, myalpha )

Array(55.43591648, dtype=float64)

In [69]:
gradcost = grad(cost, 0)

In [70]:
# adjoint method
rho = 1
def adjgrad(f, a0, alpha):
    a = [a0]
    allevals = []
    allevecs = []
    for k in range(numsteps):
        thisham = h0 + f[k]*m
        evals, evecs = jnp.linalg.eigh(thisham)
        allevals.append( evals )
        allevecs.append( evecs )
        a.append( evecs @ jnp.diag(jnp.exp(-1j*dt*evals)) @ evecs.conj().T @ a[k] )
    
    # forward trajectory
    a = jnp.stack(a, axis=0)
    print(a.shape)
    
    # initialize lambda
    resid = a[-1] - alpha
    # we are storing "lambda conjugate" throughout this calculation
    alllamb = [rho*resid.conj()]
    for k in range(numsteps-1,-1,-1):
        kk = (numsteps-1) - k
        alllamb.append( alllamb[kk] @ allevecs[k] @ jnp.diag(jnp.exp(-1j*dt*allevals[k])) @ allevecs[k].conj().T )
    
    # backward trajectory
    alllamb = jnp.flipud(jnp.stack(alllamb, axis=0))
    
    # set up mask
    mask = jnp.ones((n, n)) - jnp.eye(n)
    
    # first critical calculation
    allexpderivs = []
    for k in range(numsteps):
        amat = allevecs[k].conj().T @ m @ allevecs[k]
        dvec = -1j*dt*allevals[k]
        dvec1, dvec2 = jnp.meshgrid(dvec, dvec)
        mask = jnp.ones((n,n)) - jnp.eye(n)
        numer = jnp.exp(dvec1) - jnp.exp(dvec2)
        denom = (dvec1 - dvec2)*mask + jnp.eye(n)
        derivmat = mask*numer/denom + jnp.diag(jnp.exp(dvec))
        qmat = allevecs[k] @ (amat * derivmat) @ allevecs[k].conj().T
        allexpderivs.append( qmat )
    
    # output gradient we want
    ourgrad = []
    for k in range(numsteps):
        # because we have stored "lambda conjugate", we just need transpose here to get the dagger
        ourgrad.append( alllamb[k+1].T @ allexpderivs[k] @ a[k] )
    
    return f + jnp.real((-1j*dt)*jnp.array(ourgrad))

# simple finite-difference gradient checker
def fdgrad(kk, a0, alpha, myeps=1e-6):
    pertvec = jnp.eye(numsteps)[kk]
    cplus = cost( ftest + myeps*pertvec, a0, alpha )
    cminus = cost( ftest - myeps*pertvec, a0, alpha )
    return (cplus - cminus)/(2*myeps)

In [71]:
# UNIT TESTING for DERIVATIVE OF MATRIX EXPONENTIAL
#
# dvec = jnp.array([2.0, 4.5])
# dvec1, dvec2 = jnp.meshgrid(dvec, dvec)
# mask = jnp.ones((n,n)) - jnp.eye(n)
# numer = jnp.exp(dvec1) - jnp.exp(dvec2)
# denom = (dvec1 - dvec2)*mask + jnp.eye(n)
# test = mask*numer/denom + jnp.diag(jnp.exp(dvec))
# print(test)
# 
# npres = np.zeros((n,n))
# for i in range(n):
#     for l in range(n):
#         if dvec[i]==dvec[l]:
#             npres[i,l] = np.exp(dvec[i])
#         else:
#             npres[i,l] = (np.exp(dvec[i]) - np.exp(dvec[l]))/(dvec[i] - dvec[l])
# 
# print(npres)

In [72]:
ftest = jnp.array(np.random.normal(size=numsteps))
adjres = adjgrad( ftest, mya0, myalpha )
# print(adjres)
jaxres = gradcost( ftest, mya0, myalpha )
# print(jaxres)
print(jnp.mean(jnp.abs(adjres - jaxres)))

(101, 10)
7.528699885739343e-16


In [73]:
def myexp(x):
    thisham = h0 + x*m
    evals, evecs = jnp.linalg.eigh(thisham)
    out = evecs @ jnp.diag(jnp.exp(evals)) @ evecs.conj().T
    outflat = out.reshape((-1))
    return jnp.stack([outflat.real, outflat.imag],axis=0)

In [74]:
dmyexp = jacobian(myexp)

In [75]:
ddmyexp = jacobian(dmyexp)

In [92]:
mask = jnp.ones((n,n),dtype=jnp.complex128) - jnp.eye(n,dtype=jnp.complex128)

def myhess(x):
    z = h0 + x*m
    evals, evecs = jnp.linalg.eigh(z)
    a = evecs.conj().T @ m @ evecs
    expevals = jnp.exp(evals)
    evals1, evals2 = jnp.meshgrid(evals, evals, indexing='ij')
    expevals1, expevals2 = jnp.meshgrid(expevals, expevals, indexing='ij')
    # first D_{ii}=D_{kk} term
    diagterm1 = expevals1*jnp.diag(jnp.diag(a*a))
    # second D_{ii}=D_{kk} term
    numer1 = -expevals1 + evals1*expevals1 - evals2*expevals1 + expevals2
    denom1 = (evals1-evals2)**2 + jnp.eye(n)
    maska = mask * a
    diagterm2 = jnp.eye(n,dtype=jnp.complex128) * 2*((numer1/denom1 * maska) @ maska)
    # first D_{ii}!=D_{kk} term
    frac1 = numer1/denom1 * mask
    term1 = frac1*2*(jnp.diag(a)*a).conj().T
    # second D_{ii}!=D_{kk} term
    numer2 = -expevals1 + evals1*expevals2 - evals2*expevals2 + expevals2
    denom2 = (evals1-evals2)**2 + jnp.eye(n,dtype=jnp.complex128)
    frac2 = numer2/denom2 * mask
    term2 = frac2*2*(a*jnp.diag(a))
    # third D_{ii}!=D_{kk} term
    matij = mask*(1.0/((evals1-evals2) + jnp.eye(n,dtype=jnp.complex128)))
    matind1a = (expevals1 * matij) * a
    matind2a = (expevals2 * matij) * a
    term3 = 2*mask*((matind2a) @ (matij*a))
    term3 -= 2*matij*( matind1a @ maska )
    term3 -= 2*matij*( maska @ matind2a ) 
    # put it all together
    udagru = term1 - term2 - term3 + diagterm1 + diagterm2
    return evecs @ udagru @ evecs.conj().T

In [108]:
myx = -0.03
jaxres = ddmyexp(myx)
jaxres = (jaxres[0] + 1j*jaxres[1]).reshape((n,n))

print( jnp.mean(jnp.abs(myhess(myx) - jaxres)) )

# simple finite-difference gradient checker
def fdhess(x, myeps=1e-6):
    cplus = dmyexp( x + myeps )
    cminus = dmyexp( x - myeps )
    return (cplus[0]+1j*cplus[1] - cminus[0]-1j*cminus[1])/(2*myeps)

print( jnp.mean(jnp.abs(myhess(myx) - fdhess(myx).reshape((n,n)))) )

1.6809357684076809e-13
1.0449085200834091e-08


In [24]:
jmyhess = jit(myhess)

In [25]:
jddmyexp = jit(ddmyexp)

In [26]:
import time

In [31]:
start = time.time()
myres = jmyhess(-0.123)
end = time.time()
print(end-start)

0.0007960796356201172


In [32]:
start = time.time()
myres = jddmyexp(-0.123)
end = time.time()
print(end-start)

0.0006320476531982422
